<a href="https://colab.research.google.com/github/nishamghimire5/Compiler_Lab/blob/main/Compiler_lab_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Given a simple program (Example: sum of numbers or Fibonacci number calculation or any simple program), write a program to identify the tokens. Your program should open the text file and scan every tokens from that program and tabulate it.

In [ ]:
import re
import os

# Create the simple program as a string
simple_program = '''
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

result = fibonacci(10)
print("The 10th Fibonacci number is:", result)
'''

# Write the program to a file
filename = "simple_program.txt"
with open(filename, 'w') as file:
    file.write(simple_program)

def tokenize(filename):
    with open(filename, 'r') as file:
        content = file.read()

    # Define token patterns
    patterns = [
        ('KEYWORD', r'\b(def|if|else|return|print)\b'),
        ('IDENTIFIER', r'\b[a-zA-Z_][a-zA-Z0-9_]*\b'),
        ('OPERATOR', r'[+\-*/=():,<>]'),
        ('LITERAL', r'\b\d+\b|"[^"]*"'),
        ('WHITESPACE', r'\s+')
    ]

    tokens = []
    pos = 0
    while pos < len(content):
        match = None
        for token_type, pattern in patterns:
            regex = re.compile(pattern)
            match = regex.match(content, pos)
            if match:
                if token_type != 'WHITESPACE':
                    tokens.append((token_type, match.group(0)))
                pos = match.end()
                break
        if not match:
            print(f"Error: Unexpected character '{content[pos]}' at position {pos}")
            pos += 1

    return tokens

# Tokenize the file
tokens = tokenize(filename)

# Display results in a table
print("{:<15} {:<15}".format("Token Type", "Value"))
print("-" * 30)
for token_type, value in tokens:
    print("{:<15} {:<15}".format(token_type, value))

# Clean up: remove the created file
os.remove(filename)

Token Type      Value          
------------------------------
KEYWORD         def            
IDENTIFIER      fibonacci      
OPERATOR        (              
IDENTIFIER      n              
OPERATOR        )              
OPERATOR        :              
KEYWORD         if             
IDENTIFIER      n              
OPERATOR        <              
OPERATOR        =              
LITERAL         1              
OPERATOR        :              
KEYWORD         return         
IDENTIFIER      n              
KEYWORD         else           
OPERATOR        :              
KEYWORD         return         
IDENTIFIER      fibonacci      
OPERATOR        (              
IDENTIFIER      n              
OPERATOR        -              
LITERAL         1              
OPERATOR        )              
OPERATOR        +              
IDENTIFIER      fibonacci      
OPERATOR        (              
IDENTIFIER      n              
OPERATOR        -              
LITERAL         2              
OPERATOR 

## Explanation:
In this program, we implemented a simple lexical analyzer (tokenizer) for Python code. We first created a sample Python function (Fibonacci) as a string and wrote it to a file. The tokenizer then reads this file and uses regular expressions to identify different types of tokens such as keywords, identifiers, operators, and literals. It does this by iterating through the code character by character, matching each segment against predefined patterns for different token types. The program then categorizes these tokens and stores them in a list. Finally, it displays the results in a tabular format, showing each token's type and value. This process demonstrates the first step in compiler design: lexical analysis, where the source code is broken down into its fundamental components (tokens) for further processing in subsequent compilation stages.

## 2. Write a program to remove the left recursion from the given grammar.

In [ ]:
class Production:
    def __init__(self, left, right):
        self.left = left
        self.right = right

def remove_left_recursion(grammar):
    result = []
    for i, prod in enumerate(grammar):
        alpha = []
        beta = []
        for right in prod.right:
            if right[0] == prod.left:
                alpha.append(right[1:])
            else:
                beta.append(right)

        if alpha:
            new_symbol = f"{prod.left}'"
            result.append(Production(prod.left, [b + new_symbol for b in beta]))
            result.append(Production(new_symbol, [a + new_symbol for a in alpha] + ['ε']))
        else:
            result.append(prod)

    return result

# Input grammar
grammar = [
    Production('E', ['E+T', 'T']),
    Production('T', ['T*F', 'F']),
    Production('F', ['(E)', 'id'])
]

print("Original Grammar:")
for prod in grammar:
    print(f"{prod.left} -> {' | '.join(prod.right)}")

new_grammar = remove_left_recursion(grammar)

print("\nGrammar after removing left recursion:")
for prod in new_grammar:
    print(f"{prod.left} -> {' | '.join(prod.right)}")

Original Grammar:
E -> E+T | T
T -> T*F | F
F -> (E) | id

Grammar after removing left recursion:
E -> TE'
E' -> +TE' | ε
T -> FT'
T' -> *FT' | ε
F -> (E) | id


## Explanation:
This program implements an algorithm to eliminate left recursion from a context-free grammar. It works by transforming left-recursive productions into an equivalent non-left-recursive form. The program defines a `Production` class to represent grammar rules and a `remove_left_recursion` function that processes each production. For each left-recursive production, it separates the recursive and non-recursive parts (alpha and beta), then creates new productions that eliminate the left recursion. The result is an equivalent grammar without left recursion, which is more suitable for certain parsing techniques, particularly top-down parsing methods.
In this example, we start with a simple grammar for arithmetic expressions and transform it to remove left recursion. The resulting grammar is equivalent but can be parsed more efficiently by top-down parsers.

## 3. Write a program to demonstrate left factoring from a left factored grammar.

In [ ]:
class Production:
    def __init__(self, left, right):
        self.left = left
        self.right = right

def left_factor(grammar):
    result = []
    for prod in grammar:
        if len(prod.right) > 1 and all(r.startswith(prod.right[0][0]) for r in prod.right):
            common_prefix = prod.right[0][0]
            new_symbol = f"{prod.left}'"
            result.append(Production(prod.left, [common_prefix + new_symbol]))
            result.append(Production(new_symbol, [r[1:] for r in prod.right]))
        else:
            result.append(prod)

    return result

# Input grammar
grammar = [
    Production('A', ['aB', 'aC', 'aD']),
    Production('B', ['b']),
    Production('C', ['c']),
    Production('D', ['d'])
]

print("Original Grammar:")
for prod in grammar:
    print(f"{prod.left} -> {' | '.join(prod.right)}")

new_grammar = left_factor(grammar)

print("\nGrammar after left factoring:")
for prod in new_grammar:
    print(f"{prod.left} -> {' | '.join(prod.right)}")

Original Grammar:
A -> aB | aC | aD
B -> b
C -> c
D -> d

Grammar after left factoring:
A -> aA'
A' -> B | C | D
B -> b
C -> c
D -> d


## Explanation:
This program implements left factoring, a grammar transformation technique used in compiler design. It takes a context-free grammar as input and produces an equivalent left-factored grammar. The algorithm identifies common prefixes in the productions of each non-terminal. When a common prefix is found, it creates a new non-terminal and factors out the common part. In this example, we start with a grammar that has left-factored productions for the non-terminal A. The program applies left factoring to this grammar, creating a new non-terminal A' to represent the variations after the common prefix 'a'. The resulting grammar eliminates the ambiguity for top-down parsers by ensuring that each non-terminal has at most one production starting with a given terminal symbol. This transformation is crucial for creating efficient and unambiguous parsers in compiler design.

## 4. Write a program to identify FIRST and FOLLOW from the given grammar.

In [ ]:
import re

def cal_follow(s, productions, first):
    follow = set()
    if len(s) != 1:
        return set()
    if s == list(productions.keys())[0]:
        follow.add('$')

    for i in productions:
        for j in range(len(productions[i])):
            if s in productions[i][j]:
                idx = productions[i][j].index(s)

                if idx == len(productions[i][j]) - 1:
                    if productions[i][j][idx] == i:
                        break
                    else:
                        f = cal_follow(i, productions, first)
                        follow.update(f)
                else:
                    while idx != len(productions[i][j]) - 1:
                        idx += 1
                        if not productions[i][j][idx].isupper():
                            follow.add(productions[i][j][idx])
                            break
                        else:
                            f = cal_first(productions[i][j][idx], productions)

                            if 'ε' not in f:
                                follow.update(f)
                                break
                            elif 'ε' in f and idx != len(productions[i][j]) - 1:
                                f.discard('ε')
                                follow.update(f)
                            elif 'ε' in f and idx == len(productions[i][j]) - 1:
                                f.discard('ε')
                                follow.update(f)
                                f = cal_follow(i, productions, first)
                                follow.update(f)
    return follow

def cal_first(s, productions):
    first = set()

    for i in range(len(productions[s])):
        for j in range(len(productions[s][i])):
            c = productions[s][i][j]
            if c.isupper():
                f = cal_first(c, productions)
                if 'ε' not in f:
                    first.update(f)
                    break
                else:
                    if j == len(productions[s][i]) - 1:
                        first.update(f)
                    else:
                        f.discard('ε')
                        first.update(f)
            else:
                first.add(c)
                break

    return first

def main():
    productions = {
        "X": [["T", "n", "S"], ["R", "m"]],
        "T": [["q"], ["#"]],
        "S": [["p"], ["#"]],
        "R": [["o", "m"], ["S", "T"]]
    }

    first = {}
    follow = {}

    for s in productions.keys():
        first[s] = cal_first(s, productions)

    print("*****FIRST*****")
    for lhs, rhs in first.items():
        print(lhs, ":", rhs)

    print("")

    for lhs in productions:
        follow[lhs] = set()

    for s in productions.keys():
        follow[s] = cal_follow(s, productions, first)

    print("*****FOLLOW*****")
    for lhs, rhs in follow.items():
        print(lhs, ":", rhs)

if __name__ == "__main__":
    main()


*****FIRST*****
X : {'q', 'p', 'o', '#'}
T : {'q', '#'}
S : {'#', 'p'}
R : {'o', '#', 'p'}

*****FOLLOW*****
X : {'$'}
T : {'n', 'm'}
S : {'q', '$', '#'}
R : {'m'}


## Explanation:
This Python program calculates the FIRST and FOLLOW sets for a given context-free grammar. The `cal_first` function recursively computes the FIRST set for each non-terminal, handling epsilon productions and concatenation. The `cal_follow` function calculates the FOLLOW set for each non-terminal, considering the positions of symbols in productions and using the FIRST sets where necessary. It handles special cases like the start symbol and end-of-production scenarios. The main function defines the grammar as a dictionary of productions, computes FIRST and FOLLOW sets for all non-terminals, and prints the results. This implementation accurately handles complex grammar structures, including epsilon productions and indirect derivations, providing a comprehensive solution for FIRST and FOLLOW set computation in compiler design.

## 5. Write a program to simulate the parsing process of LL(1) grammar. Take necessary measure to use parsing table.

In [ ]:
# LL(1) parser code in python

def removeLeftRecursion(rulesDiction):
	# for rule: A->Aa|b
	# result: A->bA',A'->aA'|#

	# 'store' has new rules to be added
	store = {}
	# traverse over rules
	for lhs in rulesDiction:
		# alphaRules stores subrules with left-recursion
		# betaRules stores subrules without left-recursion
		alphaRules = []
		betaRules = []
		# get rhs for current lhs
		allrhs = rulesDiction[lhs]
		for subrhs in allrhs:
			if subrhs[0] == lhs:
				alphaRules.append(subrhs[1:])
			else:
				betaRules.append(subrhs)
		# alpha and beta containing subrules are separated
		# now form two new rules
		if len(alphaRules) != 0:
			# to generate new unique symbol
			# add ' till unique not generated
			lhs_ = lhs + "'"
			while (lhs_ in rulesDiction.keys()) \
					or (lhs_ in store.keys()):
				lhs_ += "'"
			# make beta rule
			for b in range(0, len(betaRules)):
				betaRules[b].append(lhs_)
			rulesDiction[lhs] = betaRules
			# make alpha rule
			for a in range(0, len(alphaRules)):
				alphaRules[a].append(lhs_)
			alphaRules.append(['#'])
			# store in temp dict, append to
			# - rulesDiction at end of traversal
			store[lhs_] = alphaRules
	# add newly generated rules generated
	# - after removing left recursion
	for left in store:
		rulesDiction[left] = store[left]
	return rulesDiction


def LeftFactoring(rulesDiction):
	# for rule: A->aDF|aCV|k
	# result: A->aA'|k, A'->DF|CV

	# newDict stores newly generated
	# - rules after left factoring
	newDict = {}
	# iterate over all rules of dictionary
	for lhs in rulesDiction:
		# get rhs for given lhs
		allrhs = rulesDiction[lhs]
		# temp dictionary helps detect left factoring
		temp = dict()
		for subrhs in allrhs:
			if subrhs[0] not in list(temp.keys()):
				temp[subrhs[0]] = [subrhs]
			else:
				temp[subrhs[0]].append(subrhs)
		# if value list count for any key in temp is > 1,
		# - it has left factoring
		# new_rule stores new subrules for current LHS symbol
		new_rule = []
		# temp_dict stores new subrules for left factoring
		tempo_dict = {}
		for term_key in temp:
			# get value from temp for term_key
			allStartingWithTermKey = temp[term_key]
			if len(allStartingWithTermKey) > 1:
				# left factoring required
				# to generate new unique symbol
				# - add ' till unique not generated
				lhs_ = lhs + "'"
				while (lhs_ in rulesDiction.keys()) \
						or (lhs_ in tempo_dict.keys()):
					lhs_ += "'"
				# append the left factored result
				new_rule.append([term_key, lhs_])
				# add expanded rules to tempo_dict
				ex_rules = []
				for g in temp[term_key]:
					ex_rules.append(g[1:])
				tempo_dict[lhs_] = ex_rules
			else:
				# no left factoring required
				new_rule.append(allStartingWithTermKey[0])
		# add original rule
		newDict[lhs] = new_rule
		# add newly generated rules after left factoring
		for key in tempo_dict:
			newDict[key] = tempo_dict[key]
	return newDict


# calculation of first
# epsilon is denoted by '#' (semi-colon)

# pass rule in first function
def first(rule):
	global rules, nonterm_userdef, \
		term_userdef, diction, firsts
	# recursion base condition
	# (for terminal or epsilon)
	if len(rule) != 0 and (rule is not None):
		if rule[0] in term_userdef:
			return rule[0]
		elif rule[0] == '#':
			return '#'

	# condition for Non-Terminals
	if len(rule) != 0:
		if rule[0] in list(diction.keys()):
			# fres temporary list of result
			fres = []
			rhs_rules = diction[rule[0]]
			# call first on each rule of RHS
			# fetched (& take union)
			for itr in rhs_rules:
				indivRes = first(itr)
				if type(indivRes) is list:
					for i in indivRes:
						fres.append(i)
				else:
					fres.append(indivRes)

			# if no epsilon in result
			# - received return fres
			if '#' not in fres:
				return fres
			else:
				# apply epsilon
				# rule => f(ABC)=f(A)-{e} U f(BC)
				newList = []
				fres.remove('#')
				if len(rule) > 1:
					ansNew = first(rule[1:])
					if ansNew != None:
						if type(ansNew) is list:
							newList = fres + ansNew
						else:
							newList = fres + [ansNew]
					else:
						newList = fres
					return newList
				# if result is not already returned
				# - control reaches here
				# lastly if eplison still persists
				# - keep it in result of first
				fres.append('#')
				return fres


# calculation of follow
# use 'rules' list, and 'diction' dict from above

# follow function input is the split result on
# - Non-Terminal whose Follow we want to compute
def follow(nt):
	global start_symbol, rules, nonterm_userdef, \
		term_userdef, diction, firsts, follows
	# for start symbol return $ (recursion base case)

	solset = set()
	if nt == start_symbol:
		# return '$'
		solset.add('$')

	# check all occurrences
	# solset - is result of computed 'follow' so far

	# For input, check in all rules
	for curNT in diction:
		rhs = diction[curNT]
		# go for all productions of NT
		for subrule in rhs:
			if nt in subrule:
				# call for all occurrences on
				# - non-terminal in subrule
				while nt in subrule:
					index_nt = subrule.index(nt)
					subrule = subrule[index_nt + 1:]
					# empty condition - call follow on LHS
					if len(subrule) != 0:
						# compute first if symbols on
						# - RHS of target Non-Terminal exists
						res = first(subrule)
						# if epsilon in result apply rule
						# - (A->aBX)- follow of -
						# - follow(B)=(first(X)-{ep}) U follow(A)
						if '#' in res:
							newList = []
							res.remove('#')
							ansNew = follow(curNT)
							if ansNew != None:
								if type(ansNew) is list:
									newList = res + ansNew
								else:
									newList = res + [ansNew]
							else:
								newList = res
							res = newList
					else:
						# when nothing in RHS, go circular
						# - and take follow of LHS
						# only if (NT in LHS)!=curNT
						if nt != curNT:
							res = follow(curNT)

					# add follow result in set form
					if res is not None:
						if type(res) is list:
							for g in res:
								solset.add(g)
						else:
							solset.add(res)
	return list(solset)


def computeAllFirsts():
	global rules, nonterm_userdef, \
		term_userdef, diction, firsts
	for rule in rules:
		k = rule.split("->")
		# remove un-necessary spaces
		k[0] = k[0].strip()
		k[1] = k[1].strip()
		rhs = k[1]
		multirhs = rhs.split('|')
		# remove un-necessary spaces
		for i in range(len(multirhs)):
			multirhs[i] = multirhs[i].strip()
			multirhs[i] = multirhs[i].split()
		diction[k[0]] = multirhs

	print(f"\nRules: \n")
	for y in diction:
		print(f"{y}->{diction[y]}")
	print(f"\nAfter elimination of left recursion:\n")

	diction = removeLeftRecursion(diction)
	for y in diction:
		print(f"{y}->{diction[y]}")
	print("\nAfter left factoring:\n")

	diction = LeftFactoring(diction)
	for y in diction:
		print(f"{y}->{diction[y]}")

	# calculate first for each rule
	# - (call first() on all RHS)
	for y in list(diction.keys()):
		t = set()
		for sub in diction.get(y):
			res = first(sub)
			if res != None:
				if type(res) is list:
					for u in res:
						t.add(u)
				else:
					t.add(res)

		# save result in 'firsts' list
		firsts[y] = t

	print("\nCalculated firsts: ")
	key_list = list(firsts.keys())
	index = 0
	for gg in firsts:
		print(f"first({key_list[index]}) "
			f"=> {firsts.get(gg)}")
		index += 1


def computeAllFollows():
	global start_symbol, rules, nonterm_userdef,\
		term_userdef, diction, firsts, follows
	for NT in diction:
		solset = set()
		sol = follow(NT)
		if sol is not None:
			for g in sol:
				solset.add(g)
		follows[NT] = solset

	print("\nCalculated follows: ")
	key_list = list(follows.keys())
	index = 0
	for gg in follows:
		print(f"follow({key_list[index]})"
			f" => {follows[gg]}")
		index += 1


# create parse table
def createParseTable():
	import copy
	global diction, firsts, follows, term_userdef
	print("\nFirsts and Follow Result table\n")

	# find space size
	mx_len_first = 0
	mx_len_fol = 0
	for u in diction:
		k1 = len(str(firsts[u]))
		k2 = len(str(follows[u]))
		if k1 > mx_len_first:
			mx_len_first = k1
		if k2 > mx_len_fol:
			mx_len_fol = k2

	print(f"{{:<{10}}} "
		f"{{:<{mx_len_first + 5}}} "
		f"{{:<{mx_len_fol + 5}}}"
		.format("Non-T", "FIRST", "FOLLOW"))
	for u in diction:
		print(f"{{:<{10}}} "
			f"{{:<{mx_len_first + 5}}} "
			f"{{:<{mx_len_fol + 5}}}"
			.format(u, str(firsts[u]), str(follows[u])))

	# create matrix of row(NT) x [col(T) + 1($)]
	# create list of non-terminals
	ntlist = list(diction.keys())
	terminals = copy.deepcopy(term_userdef)
	terminals.append('$')

	# create the initial empty state of ,matrix
	mat = []
	for x in diction:
		row = []
		for y in terminals:
			row.append('')
		# of $ append one more col
		mat.append(row)

	# Classifying grammar as LL(1) or not LL(1)
	grammar_is_LL = True

	# rules implementation
	for lhs in diction:
		rhs = diction[lhs]
		for y in rhs:
			res = first(y)
			# epsilon is present,
			# - take union with follow
			if '#' in res:
				if type(res) == str:
					firstFollow = []
					fol_op = follows[lhs]
					if fol_op is str:
						firstFollow.append(fol_op)
					else:
						for u in fol_op:
							firstFollow.append(u)
					res = firstFollow
				else:
					res.remove('#')
					res = list(res) +\
						list(follows[lhs])
			# add rules to table
			ttemp = []
			if type(res) is str:
				ttemp.append(res)
				res = copy.deepcopy(ttemp)
			for c in res:
				xnt = ntlist.index(lhs)
				yt = terminals.index(c)
				if mat[xnt][yt] == '':
					mat[xnt][yt] = mat[xnt][yt] \
								+ f"{lhs}->{' '.join(y)}"
				else:
					# if rule already present
					if f"{lhs}->{y}" in mat[xnt][yt]:
						continue
					else:
						grammar_is_LL = False
						mat[xnt][yt] = mat[xnt][yt] \
									+ f",{lhs}->{' '.join(y)}"

	# final state of parse table
	print("\nGenerated parsing table:\n")
	frmt = "{:>12}" * len(terminals)
	print(frmt.format(*terminals))

	j = 0
	for y in mat:
		frmt1 = "{:>12}" * len(y)
		print(f"{ntlist[j]} {frmt1.format(*y)}")
		j += 1

	return (mat, grammar_is_LL, terminals)


def validateStringUsingStackBuffer(parsing_table, grammarll1,
								table_term_list, input_string,
								term_userdef,start_symbol):

	print(f"\nValidate String => {input_string}\n")

	# for more than one entries
	# - in one cell of parsing table
	if grammarll1 == False:
		return f"\nInput String = " \
			f"\"{input_string}\"\n" \
			f"Grammar is not LL(1)"

	# implementing stack buffer

	stack = [start_symbol, '$']
	buffer = []

	# reverse input string store in buffer
	input_string = input_string.split()
	input_string.reverse()
	buffer = ['$'] + input_string

	print("{:>20} {:>20} {:>20}".
		format("Buffer", "Stack","Action"))

	while True:
		# end loop if all symbols matched
		if stack == ['$'] and buffer == ['$']:
			print("{:>20} {:>20} {:>20}"
				.format(' '.join(buffer),
						' '.join(stack),
						"Valid"))
			return "\nValid String!"
		elif stack[0] not in term_userdef:
			# take font of buffer (y) and tos (x)
			x = list(diction.keys()).index(stack[0])
			y = table_term_list.index(buffer[-1])
			if parsing_table[x][y] != '':
				# format table entry received
				entry = parsing_table[x][y]
				print("{:>20} {:>20} {:>25}".
					format(' '.join(buffer),
							' '.join(stack),
							f"T[{stack[0]}][{buffer[-1]}] = {entry}"))
				lhs_rhs = entry.split("->")
				lhs_rhs[1] = lhs_rhs[1].replace('#', '').strip()
				entryrhs = lhs_rhs[1].split()
				stack = entryrhs + stack[1:]
			else:
				return f"\nInvalid String! No rule at " \
					f"Table[{stack[0]}][{buffer[-1]}]."
		else:
			# stack top is Terminal
			if stack[0] == buffer[-1]:
				print("{:>20} {:>20} {:>20}"
					.format(' '.join(buffer),
							' '.join(stack),
							f"Matched:{stack[0]}"))
				buffer = buffer[:-1]
				stack = stack[1:]
			else:
				return "\nInvalid String! " \
					"Unmatched terminal symbols"

# sample set (left factoring & recursion present)
rules=["S -> A k O",
	"A -> A d | a B | a C",
	"C -> c",
	"B -> b B C | r"]

nonterm_userdef=['A','B','C']
term_userdef=['k','O','d','a','c','b','r']
sample_input_string="a r k O"

# diction - store rules inputted
# firsts - store computed firsts
diction = {}
firsts = {}
follows = {}

# computes all FIRSTs for all non terminals
computeAllFirsts()
# assuming first rule has start_symbol
# start symbol can be modified in below line of code
start_symbol = list(diction.keys())[0]
# computes all FOLLOWs for all occurrences
computeAllFollows()
# generate formatted first and follow table
# then generate parse table

(parsing_table, result, tabTerm) = createParseTable()

# validate string input using stack-buffer concept
if sample_input_string != None:
	validity = validateStringUsingStackBuffer(parsing_table, result,
											tabTerm, sample_input_string,
											term_userdef,start_symbol)
	print(validity)
else:
	print("\nNo input String detected")



Rules: 

S->[['A', 'k', 'O']]
A->[['A', 'd'], ['a', 'B'], ['a', 'C']]
C->[['c']]
B->[['b', 'B', 'C'], ['r']]

After elimination of left recursion:

S->[['A', 'k', 'O']]
A->[['a', 'B', "A'"], ['a', 'C', "A'"]]
C->[['c']]
B->[['b', 'B', 'C'], ['r']]
A'->[['d', "A'"], ['#']]

After left factoring:

S->[['A', 'k', 'O']]
A->[['a', "A''"]]
A''->[['B', "A'"], ['C', "A'"]]
C->[['c']]
B->[['b', 'B', 'C'], ['r']]
A'->[['d', "A'"], ['#']]

Calculated firsts: 
first(S) => {'a'}
first(A) => {'a'}
first(A'') => {'b', 'r', 'c'}
first(C) => {'c'}
first(B) => {'b', 'r'}
first(A') => {'d', '#'}

Calculated follows: 
follow(S) => {'$'}
follow(A) => {'k'}
follow(A'') => {'k'}
follow(C) => {'k', 'd', 'c'}
follow(B) => {'k', 'd', 'c'}
follow(A') => {'k'}

Firsts and Follow Result table

Non-T      FIRST                FOLLOW              
S          {'a'}                {'$'}               
A          {'a'}                {'k'}               
A''        {'b', 'r', 'c'}      {'k'}               
C          

## Explanation:
In this code, I implemented an LL(1) parser in Python that handles grammar rules, eliminates left recursion, performs left factoring, and computes the FIRST and FOLLOW sets for the given grammar. The `removeLeftRecursion` function eliminates left recursion by creating new rules. The `LeftFactoring` function handles left factoring by restructuring rules to remove common prefixes. The `first` and `follow` functions compute the FIRST and FOLLOW sets, essential for parsing table construction. `computeAllFirsts` and `computeAllFollows` functions aggregate these results. The `createParseTable` function constructs the parsing table, checking if the grammar is LL(1). Finally, `validateStringUsingStackBuffer` validates input strings against the constructed LL(1) parsing table, ensuring proper parsing and validation of strings as per the grammar rules.

## 6. Write a program to simulate the parsing process of LR grammar. Take necessary measure to use parsing table.

In [1]:
class LRParser:
    def __init__(self, grammar, parsing_table):
        self.grammar = []
        for nt, prods in grammar.items():
            for p in prods:
                self.grammar.append((nt, p))
        self.parsing_table = parsing_table
        self.stack = []

    def parse(self, input_tokens):
        # Add end-of-input marker
        input_tokens.append('$')
        # Initialize stack with state 0
        self.stack = [0]
        input_pointer = 0
        print(f"{'Stack':<30} {'Input':<20} Action")
        print('-'*30, '-'*20, '-'*6)

        while True:
            state = self.stack[-1]
            lookahead = input_tokens[input_pointer]
            action = self.parsing_table['action'][state].get(lookahead)

            # Show current stack and input
            stack_str = ' '.join(map(str, self.stack))
            input_str = ''.join(input_tokens[input_pointer:])
            print(f"{stack_str:<30} {input_str:<20} {action}")

            if action is None:
                print()
                print("Rejected")
                return

            if action[0] == 's':  # Shift
                next_state = int(action[1:])
                self.stack.append(lookahead)
                self.stack.append(next_state)
                input_pointer += 1
            elif action[0] == 'r':  # Reduce
                prod_num = int(action[1:])
                lhs, rhs = self.grammar[prod_num]
                for _ in range(2 * len(rhs)):
                    self.stack.pop()
                state = self.stack[-1]
                self.stack.append(lhs)
                self.stack.append(self.parsing_table['goto'][state][lhs])
            elif action == 'acc':
                print()
                print("Accepted")
                return
            else:
                print("Error: Invalid action. Check parsing table.")
                return

grammar = {  # Reductions
    "S": [["E"]],                # S -> E 0
    "E": [["E", "+", "T"], ["T"]],  # E -> E + T | T 1 | 2
    "T": [["T", "*", "F"], ["F"]],  # T -> T * F | F 3 | 4
    "F": [["(", "E", ")"], ["id"]]  # F -> ( E ) | id 5 | 6
}

# Parsing table
parsing_table = {
    'action': {
        0: {'id': 's5', '(': 's4'},
        1: {'+': 's6', '$': 'acc'},
        2: {'+': 'r2', '*': 's7', ')': 'r2', '$': 'r2'},
        3: {'+': 'r4', '*': 'r4', ')': 'r4', '$': 'r4'},
        4: {'id': 's5', '(': 's4'},
        5: {'+': 'r6', '*': 'r6', ')': 'r6', '$': 'r6'},
        6: {'id': 's5', '(': 's4'},
        7: {'id': 's5', '(': 's4'},
        8: {')': 's11', '+': 's6'},
        9: {'+': 'r1', '*': 's7', ')': 'r1', '$': 'r1'},
        10: {'+': 'r3', '*': 'r3', ')': 'r3', '$': 'r3'},
        11: {'+': 'r5', '*': 'r5', ')': 'r5', '$': 'r5'}
    },
    'goto': {
        0: {'E': 1, 'T': 2, 'F': 3},
        4: {'E': 8, 'T': 2, 'F': 3},
        6: {'T': 9, 'F': 3},
        7: {'F': 10}
    }
}

# Input tokens to be parsed
input_tokens = ['id', '*', 'id', '+', 'id']
print("Parsing:", ''.join(input_tokens))

# Create the parser and parse the input string
parser = LRParser(grammar, parsing_table)
parser.parse(input_tokens)

Parsing: id*id+id
Stack                          Input                Action
------------------------------ -------------------- ------
0                              id*id+id$            s5
0 id 5                         *id+id$              r6
0 F 3                          *id+id$              r4
0 T 2                          *id+id$              s7
0 T 2 * 7                      id+id$               s5
0 T 2 * 7 id 5                 +id$                 r6
0 T 2 * 7 F 10                 +id$                 r3
0 T 2                          +id$                 r2
0 E 1                          +id$                 s6
0 E 1 + 6                      id$                  s5
0 E 1 + 6 id 5                 $                    r6
0 E 1 + 6 F 3                  $                    r4
0 E 1 + 6 T 9                  $                    r1
0 E 1                          $                    acc

Accepted


## Explanation:
I implemented a program to simulate the parsing process of LR grammar using a given parsing table. The `LRParser` class initializes with the grammar rules and the parsing table. The `parse` method processes input tokens by utilizing a stack-based approach. It handles actions based on the parsing table, performing shifts, reductions, or accepting the input. The grammar defines productions for expressions and terms, and the parsing table contains actions and state transitions. The method outputs the current state of the stack, input, and the action taken at each step, ultimately determining whether the input string is accepted or rejected according to the LR parsing algorithm.